In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

# Normalisation

In [2]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
import glob
import torch
import era5_data_proc
import os

RAW_ERA5_PATH = "/vol/bitbucket/bet20/dataset/era5/global_full"
# RAW_ERA5_PATH = '/work/ec249/ec249/bet20/dataset/era5/global_full'


## Checking MEPS files

In [3]:
import neural_lam.utils

stats = neural_lam.utils.load_dataset_stats("meps_example")

for k, v in stats.items():
    print(k, v.shape)

data_mean torch.Size([17])
data_std torch.Size([17])
flux_mean torch.Size([])
flux_std torch.Size([])


In [4]:
stats

{'data_mean': tensor([ 9.9905e+04,  1.0110e+05, -2.1808e+05,  3.4544e+05,  7.1781e-01,
          7.1459e-01,  2.7186e+02,  2.7192e+02,  2.3915e+02,  2.6271e+02,
          2.4146e-01,  8.8596e-01, -2.5360e+00, -3.9970e+00,  6.2696e+00,
          8.5077e+02,  5.1629e+04]),
 'data_std': tensor([2.4345e+03, 7.6800e+02, 1.0905e+05, 5.0245e+05, 1.4583e-01, 1.6231e-01,
         4.8396e+00, 4.4669e+00, 5.4922e+00, 3.9706e+00, 4.3646e+00, 6.7067e+00,
         4.2867e+00, 5.2042e+00, 3.1096e+00, 5.9784e+02, 1.0582e+03]),
 'flux_mean': tensor(911702.5625),
 'flux_std': tensor(1037960.1250)}

## Checking ERA5 Files

In [3]:
import neural_lam.utils

stats = neural_lam.utils.load_dataset_stats("era5_uk")

for k, v in stats.items():
    print(k, v.shape)
    
stats

data_mean torch.Size([48])
data_std torch.Size([48])
flux_mean torch.Size([1])
flux_std torch.Size([1])


{'data_mean': tensor([ 2.0165e+05,  1.3330e+05,  1.0130e+05,  7.0351e+04,  5.4557e+04,
          3.0270e+04,  1.4183e+04,  1.1059e+03,  3.0181e-06,  3.9068e-06,
          3.6811e-05,  3.2273e-04,  7.1151e-04,  1.9590e-03,  3.8835e-03,
          6.2932e-03,  2.1575e+02,  2.1831e+02,  2.2008e+02,  2.4069e+02,
          2.5225e+02,  2.6771e+02,  2.7597e+02,  2.8352e+02,  8.8710e+00,
          1.3517e+01,  1.6234e+01,  1.2451e+01,  1.0400e+01,  7.2419e+00,
          5.0889e+00,  1.8382e+00, -8.0580e-01, -1.6731e-01,  5.4722e-01,
          1.6034e+00,  1.8559e+00,  2.0115e+00,  2.2037e+00,  1.6394e+00,
          3.0533e-04,  2.5111e-04, -4.0914e-03, -7.0206e-03, -7.7413e-03,
         -7.5574e-03, -4.8157e-03,  2.7199e-04]),
 'data_std': tensor([3.3787e+03, 2.5270e+03, 2.5154e+03, 1.9997e+03, 1.6560e+03, 3.3821e+03,
         1.0073e+03, 9.5701e+02, 1.4693e-07, 1.5479e-06, 3.0733e-05, 2.7484e-04,
         5.8946e-04, 1.4523e-03, 1.8634e-03, 2.0277e-03, 5.9726e+00, 5.3841e+00,
         4.7269e

In [4]:
B = 4
pred_steps = 6
num_grid_nodes = 16

# dmean = stats["data_mean"]
dmean = stats["data_mean"]
dmean = dmean.repeat(B, pred_steps, num_grid_nodes, 1)

a = stats["data_mean"]
# dmean.shape[:-1]
a = a.repeat(*dmean.shape[:-1], 1)

In [5]:
a

tensor([[[[ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          [ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          [ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          ...,
          [ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          [ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          [ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04]],

         [[ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          [ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          [ 2.0165e+05,  1.3330e+05,  1.0130e+05,  ..., -7.5574e-03,
           -4.8157e-03,  2.7199e-04],
          ...,
     

In [27]:
a.shape

torch.Size([4, 24, 3705, 48])

# ERA5 Normalisation

In [6]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/2022*.nc')
nc_files.sort()
nc_files = nc_files[:12]
nc_files

['/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_04.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_05.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_06.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_07.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_08.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_09.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_10.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_11.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_12.nc']

## Testing xarray sum method

In [7]:
i = 0
xarray_sum = np.array([0]) # (N_vars, N_levels)
np_sum = np.array([0]) # (N_vars, N_levels)
total_points = 0

for i in range(len(nc_files[:3])):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    num_points = len(data['time']) * len(data["latitude"]) * len(data["longitude"])
    total_points = total_points + num_points

    sums = np.array([data[var].sum(['time', 'latitude', 'longitude']).values for var in data.data_vars])
    print(sums.shape)
    xarray_sum = xarray_sum + sums
    
    vals = data.to_array().values
    vals = np.transpose(vals, (0, 2, 1, 3, 4))
    print(vals.shape)
    np_sum = np_sum + np.sum(vals, axis=(2, 3, 4))

    print(nc_files[i])
    print(num_points)
    
np.isclose(xarray_sum, np_sum).all()

(6, 8)
(6, 8, 124, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
(6, 8)
(6, 8, 112, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
414960
(6, 8)
(6, 8, 124, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420


True

## ERA5 Normalisation from Xarray

In [33]:
data

<xarray.Dataset> Size: 176MB
Dimensions:    (time: 124, level: 8, latitude: 65, longitude: 57)
Coordinates:
  * longitude  (longitude) float32 228B 350.0 350.2 350.5 350.8 ... 3.5 3.75 4.0
  * latitude   (latitude) float32 260B 63.0 62.75 62.5 62.25 ... 47.5 47.25 47.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
  * time       (time) datetime64[ns] 992B 2022-01-01 ... 2022-01-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 29MB 1.944e+05 ... ...
    q          (time, level, latitude, longitude) float64 29MB 3.179e-06 ... ...
    t          (time, level, latitude, longitude) float64 29MB 204.4 ... 277.0
    u          (time, level, latitude, longitude) float64 29MB 33.17 ... 1.966
    v          (time, level, latitude, longitude) float64 29MB 14.32 ... -3.715
    w          (time, level, latitude, longitude) float64 29MB -0.01995 ... -...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-15 23:54:07 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [20]:
data = xr.open_dataset(nc_files[0])
data = era5_data_proc.uk_subset(data)

In [21]:
sums = np.array([data[var].mean(['time', 'latitude', 'longitude']).values for var in data.data_vars])
sums

array([[ 1.97958755e+05,  1.31680350e+05,  1.00577091e+05,
         7.00028519e+04,  5.43791013e+04,  4.10717292e+04,
         1.44162049e+04,  1.52389741e+03],
       [ 2.94639071e-06,  4.01309092e-06,  2.20040758e-05,
         2.12156402e-04,  5.10027900e-04,  8.39396016e-04,
         2.72008603e-03,  4.78776355e-03],
       [ 2.07864530e+02,  2.12179468e+02,  2.16362146e+02,
         2.38078472e+02,  2.49579674e+02,  2.58646682e+02,
         2.73031413e+02,  2.79327570e+02],
       [ 2.94798790e+01,  2.05021198e+01,  1.98122799e+01,
         1.65335557e+01,  1.48415400e+01,  1.31751040e+01,
         9.66424146e+00,  5.94820448e+00],
       [-5.37766534e+00, -6.48773053e+00, -7.24897327e+00,
        -5.26099323e+00, -4.10058931e+00, -3.13435866e+00,
        -1.08156283e+00,  9.21981106e-01],
       [ 4.37194419e-03,  6.01573242e-03,  2.74083573e-03,
         4.12605006e-03, -2.87679536e-04, -2.87704588e-03,
        -1.06245553e-03, -6.63336960e-04]])

In [22]:
# var_sums = np.array([0]) # (N_vars, N_levels)
means = []
np_means = []
total_points = 0

for i in range(len(nc_files)):
# for i in range(3):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    num_points = len(data['time']) * len(data["latitude"]) * len(data["longitude"])
    total_points = total_points + num_points

    means.append(
        np.array([data[var].mean(['time', 'latitude', 'longitude']).values for var in data.data_vars])
    ) # (N_vars, N_levels)
    
    vals = data.to_array().values # (N_vars, N_times, N_levels, N_lats, N_lons)
    test = np.mean(vals, axis=(1, 3, 4))
    np_means.append(test)
    
    print(nc_files[i])
    print(num_points)
# var_means = var_sums / total_points

/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
414960
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_04.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_05.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_06.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_07.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_08.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_09.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_10.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_11.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_12.nc
459420


In [23]:
agg_means = np.mean(np.array(means), axis=0)
agg_means

array([[ 2.01509884e+05,  1.33204078e+05,  1.01229196e+05,
         7.02902528e+04,  5.45046372e+04,  3.03095256e+04,
         1.41506705e+04,  1.07736694e+03],
       [ 3.02062742e-06,  3.90048087e-06,  3.63757484e-05,
         3.19924511e-04,  7.06502102e-04,  1.94254609e-03,
         3.86665199e-03,  6.26392880e-03],
       [ 2.15559078e+02,  2.18190623e+02,  2.19984866e+02,
         2.40557994e+02,  2.52111628e+02,  2.67534271e+02,
         2.75863557e+02,  2.83447604e+02],
       [ 9.48125544e+00,  1.38820088e+01,  1.65905461e+01,
         1.27477311e+01,  1.06547811e+01,  7.45370871e+00,
         5.27056149e+00,  1.92616828e+00],
       [-6.65902693e-01, -6.09593413e-02,  6.63770270e-01,
         1.69961038e+00,  1.93225681e+00,  2.06362877e+00,
         2.23143236e+00,  1.66920874e+00],
       [ 2.98903141e-04,  1.88096471e-04, -4.48005649e-03,
        -7.88950576e-03, -8.75949269e-03, -8.58242505e-03,
        -5.70622556e-03,  8.98931748e-08]])

In [24]:
agg_np_mean = np.mean(np.array(np_means), axis=0)
agg_np_mean

array([[ 2.01509884e+05,  1.33204078e+05,  1.01229196e+05,
         7.02902528e+04,  5.45046372e+04,  3.03095256e+04,
         1.41506705e+04,  1.07736694e+03],
       [ 3.02062742e-06,  3.90048087e-06,  3.63757484e-05,
         3.19924511e-04,  7.06502102e-04,  1.94254609e-03,
         3.86665199e-03,  6.26392880e-03],
       [ 2.15559078e+02,  2.18190623e+02,  2.19984866e+02,
         2.40557994e+02,  2.52111628e+02,  2.67534271e+02,
         2.75863557e+02,  2.83447604e+02],
       [ 9.48125544e+00,  1.38820088e+01,  1.65905461e+01,
         1.27477311e+01,  1.06547811e+01,  7.45370871e+00,
         5.27056149e+00,  1.92616828e+00],
       [-6.65902693e-01, -6.09593413e-02,  6.63770270e-01,
         1.69961038e+00,  1.93225681e+00,  2.06362877e+00,
         2.23143236e+00,  1.66920874e+00],
       [ 2.98903141e-04,  1.88096471e-04, -4.48005649e-03,
        -7.88950576e-03, -8.75949269e-03, -8.58242505e-03,
        -5.70622556e-03,  8.98931748e-08]])

In [25]:
np.isclose(agg_means, agg_np_mean).all() 

True

In [26]:
agg_means = agg_means.reshape(-1)
agg_means

array([ 2.01509884e+05,  1.33204078e+05,  1.01229196e+05,  7.02902528e+04,
        5.45046372e+04,  3.03095256e+04,  1.41506705e+04,  1.07736694e+03,
        3.02062742e-06,  3.90048087e-06,  3.63757484e-05,  3.19924511e-04,
        7.06502102e-04,  1.94254609e-03,  3.86665199e-03,  6.26392880e-03,
        2.15559078e+02,  2.18190623e+02,  2.19984866e+02,  2.40557994e+02,
        2.52111628e+02,  2.67534271e+02,  2.75863557e+02,  2.83447604e+02,
        9.48125544e+00,  1.38820088e+01,  1.65905461e+01,  1.27477311e+01,
        1.06547811e+01,  7.45370871e+00,  5.27056149e+00,  1.92616828e+00,
       -6.65902693e-01, -6.09593413e-02,  6.63770270e-01,  1.69961038e+00,
        1.93225681e+00,  2.06362877e+00,  2.23143236e+00,  1.66920874e+00,
        2.98903141e-04,  1.88096471e-04, -4.48005649e-03, -7.88950576e-03,
       -8.75949269e-03, -8.58242505e-03, -5.70622556e-03,  8.98931748e-08])

In [27]:
stats["data_mean"]

tensor([ 2.0165e+05,  1.3330e+05,  1.0130e+05,  7.0351e+04,  5.4557e+04,
         3.0270e+04,  1.4183e+04,  1.1059e+03,  3.0181e-06,  3.9068e-06,
         3.6811e-05,  3.2273e-04,  7.1151e-04,  1.9590e-03,  3.8835e-03,
         6.2932e-03,  2.1575e+02,  2.1831e+02,  2.2008e+02,  2.4069e+02,
         2.5225e+02,  2.6771e+02,  2.7597e+02,  2.8352e+02,  8.8710e+00,
         1.3517e+01,  1.6234e+01,  1.2451e+01,  1.0400e+01,  7.2419e+00,
         5.0889e+00,  1.8382e+00, -8.0580e-01, -1.6731e-01,  5.4722e-01,
         1.6034e+00,  1.8559e+00,  2.0115e+00,  2.2037e+00,  1.6394e+00,
         3.0533e-04,  2.5111e-04, -4.0914e-03, -7.0206e-03, -7.7413e-03,
        -7.5574e-03, -4.8157e-03,  2.7199e-04])

In [28]:
(agg_means - stats["data_mean"].numpy()) / agg_means * 100

array([-6.93081197e-02, -6.88674766e-02, -7.47598230e-02, -8.66123495e-02,
       -9.57264136e-02,  1.29046850e-01, -2.30681415e-01, -2.64992774e+00,
        8.51411722e-02, -1.62311529e-01, -1.19550704e+00, -8.75881857e-01,
       -7.09071132e-01, -8.49509843e-01, -4.34753629e-01, -4.67066574e-01,
       -8.62910499e-02, -5.56483345e-02, -4.10615823e-02, -5.40832011e-02,
       -5.46796109e-02, -6.58752136e-02, -3.68933204e-02, -2.37931665e-02,
        6.43692103e+00,  2.62891128e+00,  2.14757891e+00,  2.32847838e+00,
        2.39424860e+00,  2.84139347e+00,  3.44623865e+00,  4.56852699e+00,
       -2.10080118e+01, -1.74454048e+02,  1.75588547e+01,  5.66224672e+00,
        3.95326127e+00,  2.52449580e+00,  1.24473607e+00,  1.78800698e+00,
       -2.15021663e+00, -3.35032292e+01,  8.67475551e+00,  1.10136184e+01,
        1.16240590e+01,  1.19432974e+01,  1.56066918e+01, -3.02469267e+05])

In [29]:
for a in zip(agg_means, stats["data_mean"]):
    print(float(a[0]), float(a[1]))

201509.88416330004 201649.546875
133204.0782126717 133295.8125
101229.19623203143 101304.875
70290.25277312685 70351.1328125
54504.63716560256 54556.8125
30309.525597272997 30270.412109375
14150.670509629786 14183.3134765625
1077.3669364964223 1105.9163818359375
3.020627419900134e-06 3.0180556223058375e-06
3.900480868389636e-06 3.906811798515264e-06
3.63757483529083e-05 3.681062298710458e-05
0.00031992451083885404 0.0003227266715839505
0.0007065021016883368 0.0007115117041394114
0.00194254609132609 0.0019590482115745544
0.003866651989408736 0.0038834623992443085
0.006263928803257584 0.006293185520917177
215.5590784778437 215.74508666992188
218.19062278877047 218.31204223632812
219.98486604567879 220.0751953125
240.5579936293217 240.68809509277344
252.11162754408338 252.24948120117188
267.5342714810764 267.71051025390625
275.86355680531807 275.96533203125
283.4476040055602 283.5150451660156
9.481255438723215 8.870954513549805
13.882008837058352 13.51706314086914
16.59054609194403 16.234

In [30]:
np.isclose(agg_means, stats["data_mean"]).all()

False

In [31]:
agg_np_mean.reshape(*agg_np_mean.shape, 1, 1, 1).shape


(6, 8, 1, 1, 1)

In [32]:
# mean_expanded = means.
mean_squared_diff = []
# for i in range(len(nc_files)):
xarray_std = []

for i in range(1):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    vals = data.to_array().values # (N_vars, N_times, N_levels, N_lats, N_lons)
    vals = np.transpose(vals, (0, 2, 1, 3, 4)) # (N_vars, N_levels, N_times, N_lats, N_lons)
    
    # reshape expands mean to the same shape as vals
    squared_diff = (vals - agg_np_mean.reshape(*agg_np_mean.shape, 1, 1, 1)) ** 2
    mean_squared_diff.append(
        np.mean(squared_diff, axis=(2, 3, 4))
    )
    
    # xarray builtin
    xarray_std.append(
        np.array([data[var].std(['time', 'latitude', 'longitude']).values for var in data.data_vars])
    )
    
    print(nc_files[i])
    print(num_points)

xarray_std = np.array(xarray_std).reshape(-1)
mean_squared_diff = np.mean(np.array(mean_squared_diff), axis=0).reshape(-1)
std = np.sqrt(mean_squared_diff)

/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420


In [73]:
xarray_std

array([2.42893426e+03, 2.02004012e+03, 2.27114703e+03, 2.01888831e+03,
       1.79013472e+03, 1.59352790e+03, 1.24704157e+03, 1.21204680e+03,
       1.80088179e-07, 1.38068640e-06, 1.30803320e-05, 1.65772094e-04,
       3.92830501e-04, 6.52293512e-04, 1.29022468e-03, 1.24541882e-03,
       4.80195668e+00, 5.08782063e+00, 2.78234834e+00, 4.37746257e+00,
       4.69764812e+00, 4.66767743e+00, 4.61709200e+00, 2.73543601e+00,
       1.17515724e+01, 1.22231726e+01, 1.89386276e+01, 1.56661892e+01,
       1.33934577e+01, 1.15766813e+01, 9.07740745e+00, 6.68766678e+00,
       7.92179421e+00, 1.27581614e+01, 2.27154477e+01, 1.83449770e+01,
       1.50763533e+01, 1.27639231e+01, 9.61608636e+00, 7.51053249e+00,
       3.33883427e-02, 6.71157451e-02, 1.48563811e-01, 2.71747055e-01,
       3.12395083e-01, 3.29148623e-01, 3.27426073e-01, 1.23865135e-01])

In [75]:
(xarray_std - std) / xarray_std * 100

array([-7.71292382e+01, -2.52588179e+01, -4.04043416e+00, -1.00817954e+00,
       -2.45584567e-01, -5.82732849e+02, -2.24186001e+00, -6.57045175e+00,
       -8.16324846e+00, -3.32057884e-01, -4.85662839e+01, -1.92739461e+01,
       -1.18101109e+01, -9.64716840e+01, -3.37800271e+01, -5.50767428e+01,
       -8.88813003e+01, -5.47867317e+01, -6.41736958e+01, -1.49278799e+01,
       -1.36003089e+01, -1.15069444e+02, -1.73143078e+01, -8.07913455e+01,
       -9.73845141e+01, -1.37248506e+01, -1.43662643e+00, -2.87844761e+00,
       -4.77199438e+00, -1.15459845e+01, -1.10981054e+01, -1.66916946e+01,
       -1.63515766e+01, -1.19710681e+01, -5.89344267e+00, -6.95633887e+00,
       -7.70897476e+00, -7.97429732e+00, -5.76854021e+00, -4.93700602e-01,
       -7.41329113e-01, -3.76261452e-01, -1.18050925e-01, -9.77048071e-02,
       -3.67649971e-02, -1.50218121e-02, -1.00568751e-02, -1.43434897e-03])

## Xarray Normalisation

In [63]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/2023*.nc')
nc_files.sort()
nc_files = nc_files[:12]
nc_files

['/vol/bitbucket/bet20/dataset/era5/global_full/2023_01.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_02.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_03.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_04.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_05.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_06.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_07.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_08.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_09.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_10.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_11.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2023_12.nc']

In [64]:
# Load the datasets
datasets = [
    era5_data_proc.uk_subset(xr.open_dataset(file))
    for file in nc_files
]

# Concatenate the datasets along the time dimension
combined_ds = xr.concat(datasets, dim='time')

In [65]:
nc_files[0]

'/vol/bitbucket/bet20/dataset/era5/global_full/2023_01.nc'

In [69]:
datasets[1]

<xarray.Dataset> Size: 159MB
Dimensions:    (time: 112, level: 8, latitude: 65, longitude: 57)
Coordinates:
  * longitude  (longitude) float32 228B 350.0 350.2 350.5 350.8 ... 3.5 3.75 4.0
  * latitude   (latitude) float32 260B 63.0 62.75 62.5 62.25 ... 47.5 47.25 47.0
  * level      (level) int32 32B 50 150 250 400 500 700 850 1000
  * time       (time) datetime64[ns] 896B 2023-02-01 ... 2023-02-28T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 27MB 1.888e+05 ... ...
    q          (time, level, latitude, longitude) float64 27MB 3.03e-06 ... 0...
    t          (time, level, latitude, longitude) float64 27MB 197.7 ... 275.4
    u          (time, level, latitude, longitude) float64 27MB 27.8 ... -1.302
    v          (time, level, latitude, longitude) float64 27MB -1.843 ... -1.577
    w          (time, level, latitude, longitude) float64 27MB -0.004361 ... ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-14 07:42:18 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [72]:
for dataset in datasets:
    print(f"Number of levels in dataset: {len(dataset.level)}")
    print(dataset.time.values[0])
    print(dataset.level.values)


Number of levels in dataset: 8
2023-01-01T00:00:00.000000000
[  50  150  250  400  500  600  850 1000]
Number of levels in dataset: 8
2023-02-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-03-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-04-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-05-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-06-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-07-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-08-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-09-01T00:00:00.000000000
[  50  150  250  400  500  700  850 1000]
Number of levels in dataset: 8
2023-10-01T00:00:00.000000000
[  50  150  

In [57]:
combined_ds

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 1460, level: 9, latitude: 65, longitude: 57)
Coordinates:
  * longitude  (longitude) float32 228B 350.0 350.2 350.5 350.8 ... 3.5 3.75 4.0
  * latitude   (latitude) float32 260B 63.0 62.75 62.5 62.25 ... 47.5 47.25 47.0
  * level      (level) int32 36B 50 150 250 400 500 600 700 850 1000
  * time       (time) datetime64[ns] 12kB 2022-01-01 ... 2022-12-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 389MB 1.944e+05 ......
    q          (time, level, latitude, longitude) float64 389MB 3.179e-06 ......
    t          (time, level, latitude, longitude) float64 389MB 204.4 ... 289.6
    u          (time, level, latitude, longitude) float64 389MB 33.17 ... -0....
    v          (time, level, latitude, longitude) float64 389MB 14.32 ... 3.379
    w          (time, level, latitude, longitude) float64 389MB -0.01995 ... ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-15 23:54:07 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:


# Function to compute mean and standard deviation
def compute_mean_std(ds, dims):
    mean = ds.mean(dim=dims)
    std = ds.std(dim=dims)
    return mean, std

# Compute mean and standard deviation for each variable at each level
results = {}
for var in combined_ds.data_vars:
    mean, std = compute_mean_std(combined_ds[var], dims=['time', 'latitude', 'longitude'])
    results[var] = {
        'mean': mean,
        'std': std
    }

# Display results
for var, stats in results.items():
    print(f"Variable: {var}")
    print("Mean:")
    print(stats['mean'])
    print("Standard Deviation:")
    print(stats['std'])
    print("\n")


## ERA5 Normalisation with dataloader

In [ ]:
import glob
import os
import torch
from tqdm import tqdm
from neural_lam.era5_dataset import ERA5UKDataset
from neural_lam import utils

dataset = "era5_uk_full"
batch_size = 2
n_workers = 4

In [ ]:
static = utils.load_static_data(dataset)

In [ ]:
static["data_std"]

tensor([2.5464e+03, 2.0264e+03, 2.2975e+03, 2.0187e+03, 1.7620e+03, 5.8213e+03,
        1.2267e+03, 1.1808e+03, 1.6962e-07, 1.1838e-06, 1.2919e-05, 1.5098e-04,
        3.6187e-04, 9.0703e-04, 1.3290e-03, 1.1877e-03, 6.1122e+00, 5.6558e+00,
        3.5887e+00, 4.8513e+00, 5.2299e+00, 5.2967e+00, 4.2371e+00, 2.8118e+00,
        1.4652e+01, 1.2807e+01, 1.9964e+01, 1.7005e+01, 1.4406e+01, 1.1302e+01,
        9.8326e+00, 7.3897e+00, 1.1996e+01, 1.2976e+01, 2.1520e+01, 1.8191e+01,
        1.5043e+01, 1.1692e+01, 9.7814e+00, 7.3720e+00, 3.0445e-02, 6.3717e-02,
        1.3932e-01, 2.6964e-01, 3.2182e-01, 3.4959e-01, 3.4065e-01, 1.1852e-01])

In [ ]:
ds = ERA5UKDataset(
    dataset,
    split="train",
    standardize=False,
)  # Without standardization

In [ ]:
loader = torch.utils.data.DataLoader(
    ds, batch_size, shuffle=False, num_workers=n_workers
)

In [ ]:
means = []
squares = []
flux_means = []
flux_squares = []
i = 0
LIMIT = (31 * 4 + 28 * 4 + 31 * 4) // batch_size
for init_batch, target_batch, forcing_batch in tqdm(loader):
    if i == LIMIT:
        break
    i += 1
    batch = torch.cat(
        (init_batch, target_batch), dim=1
    )  # (N_batch, N_t, N_grid, d_features)
    means.append(torch.mean(batch, dim=(1, 2)))  # (N_batch, d_features,)
    squares.append(
        torch.mean(batch**2, dim=(1, 2))
    )  # (N_batch, d_features,)


mean = torch.mean(torch.cat(means, dim=0), dim=0)  # (d_features)

 25%|██▌       | 180/716 [00:10<00:32, 16.52it/s]


# Test MEPS dataset

In [ ]:
import glob
import os
import torch
from neural_lam.weather_dataset import WeatherDataset

dataset_name = "meps_example"
batch_size = 2
n_workers = 4

train_set = WeatherDataset(
    dataset_name,
    split="train",
)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

In [ ]:
print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)

2


In [ ]:
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

torch.Size([1, 3705, 48])
torch.Size([2, 19, 63784, 17])
torch.Size([6, 3705, 12])


# Test ERA5 dataset

## Train Split

In [ ]:
import glob
import os
import torch
from neural_lam.era5_dataset import ERA5UKDataset

dataset_name = "era5_uk_full"
batch_size = 2
n_workers = 4


In [ ]:
train_set = ERA5UKDataset(
    dataset_name,
    pred_length=1,
    split="train",
    standardize=False,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)
init_states

729


tensor([[[[ 1.9736e+05,  1.3203e+05,  1.0197e+05,  ..., -2.6944e-01,
           -2.1301e-01, -1.2030e-02],
          [ 1.9744e+05,  1.3210e+05,  1.0204e+05,  ..., -2.9039e-01,
           -1.7324e-01, -3.4319e-03],
          [ 1.9754e+05,  1.3218e+05,  1.0212e+05,  ..., -2.5708e-01,
           -6.5231e-02,  1.8601e-02],
          ...,
          [ 2.0049e+05,  1.3320e+05,  1.0142e+05,  ...,  4.6545e-02,
            4.2540e-01, -9.6399e-02],
          [ 2.0052e+05,  1.3320e+05,  1.0142e+05,  ...,  1.3091e-01,
            2.2012e-01, -3.8899e-02],
          [ 2.0055e+05,  1.3319e+05,  1.0142e+05,  ...,  1.5402e-01,
           -2.2430e-01,  9.8133e-02]],

         [[ 1.9758e+05,  1.3228e+05,  1.0215e+05,  ..., -1.1360e-01,
            1.1694e-01, -1.1252e-01],
          [ 1.9765e+05,  1.3235e+05,  1.0221e+05,  ..., -9.8804e-03,
            1.9138e-02, -1.3778e-01],
          [ 1.9772e+05,  1.3242e+05,  1.0227e+05,  ...,  7.9325e-02,
           -6.4693e-02, -1.1628e-01],
          ...,
     

In [ ]:
train_set = ERA5UKDataset(
    dataset_name,
    pred_length=1,
    split="train",
    standardize=True,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)
init_states

729


tensor([[[[ 2.9441,  2.0760,  1.3319,  ...,  0.7017,  0.3570, -0.1295],
          [ 2.9493,  2.1082,  1.3931,  ...,  0.7084,  0.3536, -0.0852],
          [ 2.9531,  2.1405,  1.4528,  ...,  0.6300,  0.3673, -0.0114],
          ...,
          [ 2.9980,  3.2260,  2.6165,  ...,  0.7734,  0.6669, -0.6510],
          [ 2.9916,  3.2405,  2.6165,  ...,  1.1254,  0.8740, -0.1344],
          [ 2.9865,  3.2550,  2.6151,  ...,  1.1254,  0.8997,  1.0021]],

         [[ 2.9018,  2.1098,  1.4884,  ..., -2.6577,  0.3793, -0.2131],
          [ 2.9095,  2.1502,  1.5538,  ..., -3.5968,  0.0506, -0.4788],
          [ 2.9159,  2.1921,  1.6164,  ..., -3.8821, -0.5536, -0.5477],
          ...,
          [ 2.9005,  3.1873,  2.5810,  ..., -0.1290,  0.9322, -0.0507],
          [ 2.8941,  3.1970,  2.5781,  ...,  0.0828,  1.2078,  0.2100],
          [ 2.8877,  3.2099,  2.5753,  ...,  0.1179,  1.2232,  1.0464]]],


        [[[ 3.1045,  1.8560,  0.5640,  ..., -0.2758, -0.7146, -0.1385],
          [ 3.1032,  1.8738,

In [ ]:
path = "/vol/bitbucket/bet20/neural-lam/data/era5_uk_full/samples/train"
files = glob.glob(os.path.join(path, "*.npy"))
len(files)

1460

In [ ]:
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

torch.Size([2, 2, 3705, 48])
torch.Size([2, 1, 3705, 48])
torch.Size([2, 1, 3705, 0])


In [ ]:
val_set = ERA5UKDataset(
    dataset_name,
    pred_length=28,
    split="val",
)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

valiter = iter(val_loader)
init_states, target_states, forcing = next(valiter)

print(len(val_set))
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

376
torch.Size([2, 2, 3705, 48])
torch.Size([2, 28, 3705, 48])
torch.Size([2, 28, 3705, 0])


## Train Val Split

In [ ]:
import os
import glob
import datetime

In [ ]:
# load time step data
sample_dir_path = "data/era5_uk/samples/train"
sample_files = glob.glob(f'{sample_dir_path}/*.npy')
sample_files.sort()
sample_files = [os.path.basename(f)[:-4] for f in sample_files]
sample_times = [datetime.datetime.strptime(f, '%Y%m%d%H%M%S') for f in sample_files]

In [ ]:
sample_files[-1]

'20221231180000'

In [ ]:
for i in range(1, len(sample_times)):
    delta = sample_times[i] - sample_times[i-1]
    if delta != datetime.timedelta(hours=6):
        print(f"Missing time step: {sample_times[i-1]} -> {sample_times[i]} ({delta})")

In [ ]:
sample_dir_path = "data/era5_uk/samples/val"
# directories = [name for name in os.listdir(sample_dir_path)]
# directories.sort()
# directories
val_months = ["01", "04", "07", "10"]

In [ ]:
month_path = os.path.join(sample_dir_path, val_months[1])
print(month_path)
sample_files = glob.glob(f'{month_path}/*.npy')
sample_files.sort()
sample_files

data/era5_uk_full/samples/val/04


['data/era5_uk_full/samples/val/04/20230401000000.npy',
 'data/era5_uk_full/samples/val/04/20230401060000.npy',
 'data/era5_uk_full/samples/val/04/20230401120000.npy',
 'data/era5_uk_full/samples/val/04/20230401180000.npy',
 'data/era5_uk_full/samples/val/04/20230402000000.npy',
 'data/era5_uk_full/samples/val/04/20230402060000.npy',
 'data/era5_uk_full/samples/val/04/20230402120000.npy',
 'data/era5_uk_full/samples/val/04/20230402180000.npy',
 'data/era5_uk_full/samples/val/04/20230403000000.npy',
 'data/era5_uk_full/samples/val/04/20230403060000.npy',
 'data/era5_uk_full/samples/val/04/20230403120000.npy',
 'data/era5_uk_full/samples/val/04/20230403180000.npy',
 'data/era5_uk_full/samples/val/04/20230404000000.npy',
 'data/era5_uk_full/samples/val/04/20230404060000.npy',
 'data/era5_uk_full/samples/val/04/20230404120000.npy',
 'data/era5_uk_full/samples/val/04/20230404180000.npy',
 'data/era5_uk_full/samples/val/04/20230405000000.npy',
 'data/era5_uk_full/samples/val/04/2023040506000

In [ ]:
import shutil
import glob
import os

# Specify the source and destination directories
src_dir = 'data/era5_uk_full/samples/train/'
dst_dir = 'data/era5_uk_full/samples/val/01'

os.makedirs(dst_dir, exist_ok=True)

# Use glob to get all .npy files that start with 2023 in the source directory
files = glob.glob(os.path.join(src_dir, '202301*.npy'))

# print(files)
# print(len(files))

# Move each file to the destination directory
for file in files:
    shutil.move(file, dst_dir)

In [ ]:
31 * 4

124

In [ ]:

pred_length = 28
sample_length = pred_length + 2
val_samples = []
sample_dir_path = "./data/era5_uk_full/samples/val"
month_samples = {}

for month in val_months:
    month_dir = os.path.join(sample_dir_path, month)
    data_files = glob.glob(os.path.join(month_dir, "*.npy"))
    data_files.sort()
    month_samples[month] = data_files
    n_samples = len(data_files) - sample_length + 1
    for i in range(n_samples):
        val_samples.append((month, i))
    

N = len(val_samples)
N

376

In [ ]:
idx = 332
month, idx = val_samples[idx]
sample_names = month_samples[month]

sample_names[idx], idx

('./data/era5_uk_full/samples/val/10/20231013180000.npy', 51)

In [ ]:
# verify number of time steps
val_files = glob.glob(f'{src_dir}/10/*.npy')
len(val_files)

124

In [ ]:
val_files = glob.glob(f'{src_dir}/*.npy')
# val_files.sort()
# wanted = [v for v in val_files if "202301" in v or "202310" in v or "202304" in v or "202307" in v]
# wanted

[]